# DeepFM




DeepFM model:FM + DNN

<img src="../data/img/DeepFM_model.png" height="600" width="600" style="float:left" >




In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.model_selection import train_test_split

In [7]:
class DeepFM(nn.Module):
    
    def __init__(self, input_size, num_factors, dims, num_layers):
        super(DeepFM, self).__init__()
        self.V = nn.Embedding(num_embeddings=input_size, embedding_dim=num_factors)
        self.X = nn.Embedding(num_embeddings=input_size, embedding_dim=1)
        self.linear = nn.Linear(in_features=1, out_features=1, bias=True)
        self.input_dim = dim*num_factors
        self.mlp = nn.Sequential()
        for i in range(num_layers-1):
            self.mlp.add_module('layer'+str(i+1),nn.Linear(self.input_dim, self.input_dim))
            self.mlp.add_module('relu'+str(i+1),nn.ReLU())
        self.mlp.add_module('layer'+str(num_layers),nn.Linear(self.input_dim,1))
        
    def forward(self, x):
        v = self.V(x)
        fm_output = self.linear(0.5*((v.sum(dim=1))**2 - (v**2).sum(dim=1)).sum(dim=1, keepdim=True) + self.X(x).sum(dim=1))
        nn_input = v.reshape(-1, self.input_dim)
        nn_output = self.mlp(nn_input)
        y = torch.sigmoid(fm_output+nn_output)
        return y 

In [11]:
input_size = 10000
num_factors = 10
dims=2
num_layers = 3

In [12]:
model = DeepFM(input_size, num_factors, dims, num_layers)

In [13]:
print(model)

DeepFM(
  (V): Embedding(10000, 10)
  (X): Embedding(10000, 1)
  (linear): Linear(in_features=1, out_features=1, bias=True)
  (mlp): Sequential(
    (layer1): Linear(in_features=20, out_features=20, bias=True)
    (relu1): ReLU()
    (layer2): Linear(in_features=20, out_features=20, bias=True)
    (relu2): ReLU()
    (layer3): Linear(in_features=20, out_features=1, bias=True)
  )
)
